In [2]:
!pwd

/Users/debarchan/PycharmProjects/Adenocarcinoma-Detection/research


In [3]:
import os 

os.chdir("../")

In [4]:
!pwd

/Users/debarchan/PycharmProjects/Adenocarcinoma-Detection


In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    '''this class object (entity) will be returned by ingestion component'''
    # same variables defined in config.yaml
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from carcinomaDetection.constants import *
from carcinomaDetection.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config 

In [8]:
import os 
import zipfile
import gdown
from carcinomaDetection import logger
from carcinomaDetection.utils.common import get_size

In [9]:
## Create data ingestion component 

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    def download_file(self) -> str:
        '''fetch data from the url'''
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")
            
            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)
            
            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e

    def extract_zip_file(self):
        '''extracts zip file into the data directory'''
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-06-06 23:42:56,180: INFO: common: yaml file: config/config.yaml loaded successfully!]
[2024-06-06 23:42:56,183: INFO: common: yaml file: params.yaml loaded successfully!]
[2024-06-06 23:42:56,189: INFO: common: created directory at: artifacts]
[2024-06-06 23:42:56,209: INFO: common: created directory at: artifacts/data_ingestion]
[2024-06-06 23:42:56,227: INFO: 1935380557: Downloading data from https://drive.google.com/file/d/1QtnMBvt2zue-BicSfNVNUxRpwAHdTVyu/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1QtnMBvt2zue-BicSfNVNUxRpwAHdTVyu
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1QtnMBvt2zue-BicSfNVNUxRpwAHdTVyu&confirm=t&uuid=470e4f57-6f88-45f2-a7b9-396df8a59799
To: /Users/debarchan/PycharmProjects/Adenocarcinoma-Detection/artifacts/data_ingestion/data.zip
100%|██████████| 125M/125M [00:04<00:00, 27.0MB/s] 

[2024-06-06 23:43:02,665: INFO: 1935380557: Downloaded data from https://drive.google.com/file/d/1QtnMBvt2zue-BicSfNVNUxRpwAHdTVyu/view?usp=sharing into file artifacts/data_ingestion/data.zip]
